In [14]:
# to do
# consider the case when there is no vintage date appended to a variable
# fill NaN using the values from the last vintage date

In [15]:
vintage_date = '2011-04-01' # until the end of day
start_quarter = '1980Q4' # inclusive
end_quarter = '2010Q2' # inclusive

fillna_from_last_vintage = False
variables = {
    'raw_variables': set(['GDPC1', ]),
    'raw_variables_transform': set([]),
    'observed_variables': set([]),
}

In [16]:
import pathlib, warnings
import pandas as pd
import numpy as np
import datetime as datetime

In [17]:
data_path = pathlib.Path('raw_variables')
assert data_path.exists()

In [18]:
vintage_date = pd.to_datetime(vintage_date)
start_observation = pd.to_datetime(start_quarter).to_period('Q').start_time
end_observation = pd.to_datetime(end_quarter).to_period('Q').end_time

if vintage_date < end_observation:
    warnings.warn('Your vintage date is smaller than the last observation date, so the last observation date will be the vintage date.')
    end_observation = vintage_date

In [19]:
description_obs = pd.read_excel('observed_variable_description.xlsx')
description_raw = pd.read_csv('raw_variable_description.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 90: invalid start byte

### generate observables

In [7]:
dict_obs_raw = dict()

if len(variables['observed_variables']) > 0:

    # identify corresponding raw variables and store their names in 'raw_variables_tranform'
    set_raw_variables = {data_file.stem for data_file in data_path.glob('*.*')}

    for _, row in description_obs.iterrows():
        if row['id'] in variables['observed_variables']:

            set_raw = set()
            for raw_variable in set_raw_variables:
                if raw_variable in row['construction']:

                    set_raw.update({raw_variable})
                    variables['raw_variables_transform'].update({raw_variable})
            
            dict_obs_raw[row['id']] = set_raw

    # description_obs = description_obs[description_obs['id'].map(lambda x: x in variables['observed_variables'])]
    
    # idenfity_string = ' '.join(description_obs['construction'].values)
    # for variable in set_raw_variables:
    #     if variable in idenfity_string:
    #         dict_obs_raw[]
    #         variables['raw_variables_transform'].update({variable})

In [8]:
def generate_raw_variables(variables):

    index = -1

    for variable in variables:

        to_merge = False

        for data_file in set(data_path.glob('*.*')):

            if data_file.stem == variable:

                # load dataset
                data_set = pd.read_csv(data_file, index_col=0)
                data_set.index = pd.to_datetime(data_set.index)

                # load frequency of the variable
                frequency = description_raw[description_raw['id']==variable]['frequency_short'].values[0]

                # choose data within observation period
                observation_dates = data_set.index.map(lambda x: start_observation <= x <= end_observation)

                if len(observation_dates.values) == 0:
                    warnings.warn(f'\n{variable} has no value within the observation period you choose for any vintage date!\n')
                    break

                # choose data within vintage date
                vintage_column = ''

                if frequency == 'D':
                    vintage_column = data_set.columns.values[-1]
                else:
                    for column in data_set.columns.values:
                        if int(vintage_date.strftime('%Y%m%d')) >= int(column[-8:]):
                            vintage_column = column
                        else:
                            break

                if vintage_column == '':
                    warnings.warn(f'\nFor {variable}, the vintage date you choose is out of bound!\n')
                    break

                # combine desired observation period and vintage date
                # for daily and monthly data, take the average over the quarter
                if len(observation_dates.values) > 0 and vintage_column != '':

                    data_set['quarter'] = data_set.index.to_period('Q').values
                    data_set = data_set[observation_dates][[vintage_column, 'quarter']].copy()

                    if np.sum(~np.isnan(data_set[vintage_column].values)) == 0:
                        warnings.warn(f'\n{variable} has no value in the observation period and vintage date you choose.\n')
                        break
                    else:
                        index += 1
                        to_merge = True
                        data_to_merge = data_set.groupby('quarter').mean()

                break

        if to_merge == True:

            if index == 0:
                data_output = data_to_merge.copy()
            else:
                data_output = pd.merge(data_output, data_to_merge, how='outer', left_index=True, right_index=True, sort=True)

    return data_output

In [9]:
df_raw = generate_raw_variables(variables['raw_variables'])
df_for_obs = generate_raw_variables(variables['raw_variables_transform'])

UnboundLocalError: local variable 'data_output' referenced before assignment

In [10]:
dict_raw = dict()
for column in df_for_obs.columns.values:
    if column[-8:].isdigit():
        dict_raw[column[:-9]] = column
    else:
        dict_raw[column] = column

NameError: name 'df_for_obs' is not defined

In [11]:
df = df_raw.copy()
assert df_raw.shape[0] == df_for_obs.shape[0]

NameError: name 'df_for_obs' is not defined

In [12]:
for observable in variables['observed_variables']:

    vintage_date_observable = 99999999
    for raw_variable in dict_obs_raw[observable]:
        vintage_date_observable = min(vintage_date_observable, int(dict_raw[raw_variable][-8:]))
    column_name = observable + '_' + str(vintage_date_observable)

    if observable == 'xgdp_q_obs':
        # ΔLN(GDPC1)*100
        df.loc[:, column_name] = np.log(df_for_obs[dict_raw['GDPC1']].values/df_for_obs[dict_raw['GDPC1']].shift().values)*100

    elif observable == 'pgdp_q_obs':
        # ΔLN(GDPCTPI)*100
        df.loc[:, column_name] = np.log(df_for_obs[dict_raw['GDPCTPI']].values/df_for_obs[dict_raw['GDPCTPI']].shift().values)*100

    elif observable == 'rff_q_obs':
        # DFF/4
        df.loc[:, column_name] = df_for_obs[dict_raw['DFF']]/4

    elif observable == 'fpi_q_obs':
        # ΔLN(FPI/GDPCTPI)*100
        df.loc[:, column_name] = np.log((df_for_obs[dict_raw['FPI']].values/df_for_obs[dict_raw['GDPCTPI']].values)/(df_for_obs[dict_raw['FPI']].shift().values/df_for_obs[dict_raw['GDPCTPI']].shift().values))*100

    elif observable == 'pcer_q_obs':
        # ΔLN(PCE/GDPCTPI)*100
        df.loc[:, column_name] = np.log((df_for_obs[dict_raw['PCE']].values/df_for_obs[dict_raw['GDPCTPI']].values)/(df_for_obs[dict_raw['PCE']].shift().values/df_for_obs[dict_raw['GDPCTPI']].shift().values))*100

    elif observable == 'cp_q_obs':
        # (DBAA-DGS10)/4
        df.loc[:, column_name] = (df_for_obs[dict_raw['DBAA']] - df_for_obs[dict_raw['DGS10']])/4

    elif observable == 'wage_obs':
        # ΔLN(COMPNFB/GDPCTPI)*100
        df.loc[:, column_name] = np.log((df_for_obs[dict_raw['COMPNFB']].values/df_for_obs[dict_raw['GDPCTPI']].values)/(df_for_obs[dict_raw['COMPNFB']].shift().values/df_for_obs[dict_raw['GDPCTPI']].shift().values))*100

    else:
        warnings.warn(f'\n{observable} is not exported as an osbervable.\n')


In [13]:
df.to_csv(f"data_{vintage_date.strftime('%Y%m%d')}.csv")